# New Bachata Playlist

In [1]:
import csv
from datetime import datetime, timedelta

import spotipy
from spotipy.oauth2 import SpotifyOAuth

## Get the date for last Friday

In [2]:
# Get the current date
today = datetime.today().date()

# Calculate the offset to the second most recent Friday
days_back = ((today.weekday() - 4) % 7) + 7

# Subtract the offset to get the last Friday
threshold_date = today - timedelta(days=days_back)

print(f"The most recent Friday was on {threshold_date}.")

The most recent Friday was on 2024-05-31.


## Authenticate with Spotify

In [3]:
client_id = "f412462e4440495d88e95d8288012f57"
client_secret = "fbcbcd663d664bf1b2cc1f2cbdd90024"

scope = "user-library-read playlist-read-private playlist-read-collaborative playlist-modify-public"
auth_manager = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://localhost:8888/callback",
    scope=scope,
)
sp = spotipy.Spotify(auth_manager=auth_manager)

## Search for bachata artists in playlists

In [22]:
search_genre = "bachata"

In [25]:
artists_list = []

# Fetch in batches
batch_size = 50  # max=50
offset = 0
while True:
    results = sp.search(
        q=f"genre:{search_genre}", type="artist", limit=batch_size, offset=offset
    )
    artists_batch = results["artists"]["items"]
    for artist_item in artists_batch:
        artist_dict = {
            "name": artist_item["name"],
            "id": artist_item["id"],
        }
        artists_list.append(artist_dict)

    if len(artists_batch) < batch_size:  # No more results to fetch
        break

    offset += batch_size

print(f"Found {len(artists_list)} artists in the genre '{search_genre}'.")

Found 211 artists in the genre 'bachata'.


In [26]:
with open(f"{search_genre}_artists.csv", "w", newline="") as f:
    dict_writer = csv.DictWriter(f, artists_list[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(artists_list)

artists_list

[{'name': 'Mr. Don', 'id': '12YP3RGVLp3c36Zi6lFRtR'},
 {'name': 'DJ Tony Pecino', 'id': '0Jc5bLNs5tbmmecIYaURND'},
 {'name': 'Jhonny Evidence', 'id': '7EndqnY5tR0pzhbVFJr5Zy'},
 {'name': 'JR', 'id': '3XH9HjOmomIwvkGpYRKIGl'},
 {'name': 'Daniel Santacruz', 'id': '4tLUnrSgMM7tT0zVs3wX61'},
 {'name': 'Karlos Rosé', 'id': '6FQPrGwEepUrGKbzD3Zeib'},
 {'name': 'Johnny Sky', 'id': '3EVZsBWq5KqdtS6eVNHN8d'},
 {'name': 'Grupo Extra', 'id': '4L5SLCIDWPPNGzrLraYCOS'},
 {'name': 'Prince Royce', 'id': '3MHaV05u0io8fQbZ2XPtlC'},
 {'name': 'Luis Segura', 'id': '3xgpQ8RHWeHY9ARr9xkOpp'},
 {'name': 'Romeo Santos', 'id': '5lwmRuXgjX8xIwlnauTZIP'},
 {'name': 'Raulin Rodriguez', 'id': '5rvaaWutd0gsbc2iMNo9k5'},
 {'name': 'Juan Luis Guerra 4.40', 'id': '3nlpTZci9O5W8RsNoNH559'},
 {'name': 'Aventura', 'id': '1qto4hHid1P71emI6Fd8xi'},
 {'name': 'Kewin Cosmos', 'id': '1nf6gLEC45OmoJFIxU7aUT'},
 {'name': 'Héctor Acosta "El Torito"', 'id': '4TCAAIMmeZXCTVlHUsicOn'},
 {'name': 'Luis Vargas', 'id': '6Mjbr8K3MiiRM

## Search for new songs by bachata artists

In [29]:
new_songs = []
for artist in artists_list[0:1]:
    print(f"Fetching albums for {artist['name']}...")
    results = sp.artist_albums(artist["id"])
    for album in results["items"]:
        release_date_str = album["release_date"]
        release_date_precision = album["release_date_precision"]
        if release_date_precision == "day":
            release_date = datetime.strptime(release_date_str, "%Y-%m-%d").date()
        elif release_date_precision == "month":
            release_date = datetime.strptime(release_date_str, "%Y-%m").date()
        elif release_date_precision == "year":
            release_date = datetime.strptime(release_date_str, "%Y").date()

        if release_date >= threshold_date:
            # TODO: Check if the album is already in the list
            # TODO: Check if the album is a compilation
            album_details = sp.album(album["id"])
            for track in album_details["tracks"]["items"]:
                song_dict = {
                    "song": track["name"],
                    "artist": track["artists"][0]["name"],
                    "album": album["name"],
                    "album_type": album["album_type"],
                    "release_date": str(release_date),
                    "id": track["id"],
                }
                new_songs.append(song_dict)
track_ids = [track["id"] for track in new_songs if track["id"] not in track_ids]

Fetching albums for Mr. Don...


In [ ]:
playlist_name = f"Automated New Bachata - {threshold_date.month:02d}-{threshold_date.day:02d} to {f"{today.month:02d}-{today.day:02d}"}"

with open(f"{playlist_name}.csv", "w", newline="") as f:
    dict_writer = csv.DictWriter(f, new_songs[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(new_songs)
    
new_songs

[{'id': '1cKuAgP3gKxj0nuQ2xqYmt',
  'song': 'Live Session',
  'artist': 'Johnny Sky',
  'album': 'Live Session',
  'album_type': 'single',
  'release_date': '2024-05-09'},
 {'id': '7dLhnNR9x9kn8EnS1WWwa9',
  'song': "Ponte Pa' Mi - Bachata Version 2.0",
  'artist': 'DJ Tony Pecino',
  'album': "Ponte Pa' Mi (Bachata Version 2.0)",
  'album_type': 'single',
  'release_date': '2024-05-10'},
 {'id': '1sI9c9rGGdYeGE6nQQCAYQ',
  'song': "Hecha Pa' Mi - Bachata Version 2.0",
  'artist': 'DJ Tony Pecino',
  'album': "Hecha Pa' Mi (Bachata Version 2.0)",
  'album_type': 'single',
  'release_date': '2024-05-03'}]

## Add songs to playlist

In [ ]:
playlist_id = "16GAR1gl8pCjmaiplCr5xr"
sp.playlist_replace_items(playlist_id, track_ids)
sp.playlist_change_details(
    playlist_id,
    name=playlist_name,
    description=f"Automated playlist with new Bachata songs from {threshold_date} to {today.date()}.",
)